## Ejercicios de pair programming 23 enero: Anova

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
import random 

# Estadísticos
# -----------------------------------------------------------------------
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("../datos/world_risk_index_sin_outliers_est.csv", index_col = 0)
df.head(2)

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
0,papua new guinea,Very High,Very High,Very High,Very High,2.906480,23.26,1.296928,1.179006,0.962932,1.537045,2011.0,0.895683
1,madagascar,Very High,Very High,Very High,Very High,2.594391,20.68,1.545395,2.260942,1.017385,0.974085,2011.0,0.792566


In [3]:
outliers = pd.read_csv("../datos/world_risk_index_outliers_est.csv", index_col = 0)
outliers.head(2)

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
0,vanuatu,Very High,Very High,High,High,1.640675,56.33,0.801253,0.792708,0.541556,0.926242,2011.0,0.563758
1,tonga,Very High,Very High,Medium,Medium,1.292570,56.04,0.376459,0.030528,0.707655,0.185736,2011.0,0.560853


### Info columnas
|Columna| Tipo de dato | Descripcion |
|-------|--------------|-------------|
|Region| String|	Name of the region.
|WRI	| Decimal |	World Risk Score of the region.
|Exposure	| Decimal |	Risk/exposure to natural hazards such as earthquakes, hurricanes, floods, droughts, and sea ​​level rise.
|Vulnerability	| Decimal |	Vulnerability depending on infrastructure, nutrition, housing situation, and economic framework conditions.
|Susceptibility	| Decimal |	Susceptibility depending on infrastructure, nutrition, housing situation, and economic framework conditions.
|Lack of Coping Capabilities	| Decimal |	Coping capacities in dependence of governance, preparedness and early warning, medical care, and social and material security.
|Lack of Adaptive Capacities| Decimal |	Adaptive capacities related to coming natural events, climate change, and other challenges.
|Year	| Decimal |	Year data is being described.
|WRI Category| String|	WRI Category for the given WRI Score.
|Exposure Category| String|	Exposure Category for the given Exposure Score.
|Vulnerability Categoy| String|	Vulnerability Category for the given Vulnerability Score.
|Susceptibility Category| String|	Susceptibility Category for the given Susceptibility Score.

Link a la base de datos : https://www.kaggle.com/datasets/tr1gg3rtrash/global-disaster-risk-index-time-series-dataset

### Nuestra variable respuesta es Exposure_Sklearn, queremos saber cual es el riesgo de desastres naturales dependiendo del resto de variables

---

### df limpio

---

In [5]:
df.columns

Index(['region', 'exposure_category', 'wri_category', 'vulnerability_category',
       'susceptibility_category', 'wri', 'exposure', 'vulnerability',
       'susceptibility', 'lack_of_coping_capabilities',
       'lack_of_adaptive_capacities', 'year', 'exposure_Sklearn'],
      dtype='object')

In [11]:
lm = ols("exposure_Sklearn ~ region  + exposure_category + wri_category + vulnerability_category + susceptibility_category + wri + vulnerability + susceptibility + lack_of_coping_capabilities + lack_of_adaptive_capacities + year", data=df).fit()
sm.stats.anova_lm(lm) #Aplicamos el ANOVA a nuestro df con la variable respuesta y las predictoras

,df,sum_sq,mean_sq,F,PR(>F)
region,164.0,5.191859e+01,3.165767e-01,2559.931104,0.000000e+00
exposure_category,4.0,2.514415e+00,6.286039e-01,5083.072487,0.000000e+00
wri_category,4.0,3.020482e-01,7.551206e-02,610.612313,3.121536e-314
vulnerability_category,4.0,9.363604e-03,2.340901e-03,18.929203,3.340962e-15
susceptibility_category,4.0,5.082138e-03,1.270535e-03,10.273910,3.289502e-08
wri,1.0,1.066009e+00,1.066009e+00,8620.057524,0.000000e+00
vulnerability,1.0,4.120953e-01,4.120953e-01,3332.322126,0.000000e+00
susceptibility,1.0,4.167684e-04,4.167684e-04,3.370110,6.658470e-02
lack_of_coping_capabilities,1.0,4.969090e-04,4.969090e-04,4.018150,4.519028e-02
lack_of_adaptive_capacities,1.0,7.594349e-08,7.594349e-08,0.000614,9.802329e-01


El DF nos indica las que son columnas categorica (*region, exposure_category, wri_category,vulnerability_category, susceptibility_category*) y numerica todas las que tienen un valor de 1.

El F evalua la capacidad que tiene cada variable predictora de influir sobre la variable respuesta. Por lo cual la que influyen mas son *wri* y *vulnerability*.  
Nos damops cuenta que *exposure_category* está calculada directamente con la categoria de *exposure* por lo cual decidimpos no incluirla en nuestro modelo de predicion.

Mirando la columna del PR(>F) podemos concluir que *susceptibility,lack_of_adaptive_capacities* son mayores de 0.05 por lo cual NO influyen sobre nuestra variable respuesta.

In [12]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       exposure_Sklearn   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     2445.
Date:                Mon, 30 Jan 2023   Prob (F-statistic):               0.00
Time:                        19:17:25   Log-Likelihood:                 5353.6
No. Observations:                1706   AIC:                        -1.033e+04
Df Residuals:                    1519   BIC:                            -9315.
Df Model:                         186                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
Intercept                                     -0.0972      0.256     -0.380      0.704      -0.598       0.404
region[T.albania]                              0.0740      0.008      8.997      0.000       0.058       0.090
region[T.algeria]                              0.0307      0.008      3.907      0.000       0.015       0.046
region[T.angola]                               0.0065      0.005      1.277      0.202      -0.003       0.016
region[T.argentina]                           -0.0336      0.009     -3.756      0.000      -0.051      -0.016
region[T.armenia]                              0.0211      0.008      2.557      0.011       0.005       0.037
region[T.australia]                            0.0950      0.011      8.885      0.000       0.074       0.116
region[T.austria]                              0.0549      0.011      5.029      0.000       0.033       0.076
region[T.azerbaijan]                           0.0161      0.008      1.990      0.047       0.000       0.032
region[T.bahamas]                             -0.0121      0.009     -1.316      0.188      -0.030       0.006
region[T.bahrain]                             -0.0385      0.009     -4.245      0.000      -0.056      -0.021
region[T.barbados]                            -0.0892      0.009     -9.438      0.000      -0.108      -0.071
region[T.belarus]                             -0.0185      0.009     -2.006      0.045      -0.037      -0.000
region[T.belgium]                              0.0228      0.011      2.150      0.032       0.002       0.044
region[T.belize]                               0.0215      0.008      2.853      0.004       0.007       0.036
region[T.benin]                               -0.0029      0.005     -0.547      0.585      -0.013       0.008
region[T.bhutan]                               0.0079      0.007      1.127      0.260      -0.006       0.022
region[T.bolivia]                              0.0053      0.007      0.762      0.446      -0.008       0.019
region[T.bosnia and herzegovina]               0.0055      0.008      0.668      0.504      -0.011       0.022
region[T.botswana]                            -0.0003      0.008     -0.043      0.966      -0.015       0.015
region[T.brazil]                              -0.0119      0.008     -1.465      0.143      -0.028       0.004
region[T.bulgaria]                            -0.0067      0.009     -0.738      0.461      -0.025       0.011
region[T.burkina faso]                        -0.0038      0.005     -0.741      0.459      -0.014       0.006
region[T.burundi]                             -0.0078      0.005     -1.506      0.132      -0.018       0.002
region[T.cameroon]                            -0.0070      0.006     -1.195      0.232

- El P>|t| de la columna *region* en algunas regiones es menor de 0.05 por lo cual nos quedamos con esta columna (la region influye sobre la variable respuesta).
- exposure_category también influye, pero ya decidimos quitarla.
- El P>|t| se ve afectado especialmente por la categoria de region al ser que tenemos unas 200 mas o menos.
- R square de nuestra variables predictoras explican un 95% de nuestra variable respuesta. 

---

### outliers

---

In [13]:
outliers.head()

,region,exposure_category,wri_category,vulnerability_category,susceptibility_category,wri,exposure,vulnerability,susceptibility,lack_of_coping_capabilities,lack_of_adaptive_capacities,year,exposure_Sklearn
0,vanuatu,Very High,Very High,High,High,1.640675,56.33,0.801253,0.792708,0.541556,0.926242,2011.0,0.563758
1,tonga,Very High,Very High,Medium,Medium,1.292570,56.04,0.376459,0.030528,0.707655,0.185736,2011.0,0.560853
2,philippines,Very High,Very High,High,High,0.725110,45.09,0.552087,0.592868,0.773824,0.106661,2011.0,0.451167
3,solomon islands,Very High,Very High,Very High,High,0.628547,36.40,1.475212,1.440562,0.987862,1.731821,2011.0,0.364119
4,guatemala,Very High,Very High,High,High,0.315014,38.42,0.588423,0.627259,0.439602,0.589349,2011.0,0.384353


In [14]:
lm_outliers = ols("exposure_Sklearn ~ region  + exposure_category + wri_category + vulnerability_category + susceptibility_category + wri + vulnerability + susceptibility + lack_of_coping_capabilities + lack_of_adaptive_capacities + year", data=outliers).fit()
sm.stats.anova_lm(lm_outliers)

,df,sum_sq,mean_sq,F,PR(>F)
region,28.0,4.081559,0.145770,2764.014789,8.662355e-208
exposure_category,1.0,0.000617,0.000617,11.693028,7.884497e-04
wri_category,3.0,0.013726,0.004575,86.757567,6.953253e-34
vulnerability_category,4.0,0.017324,0.004331,82.123763,2.051130e-38
susceptibility_category,4.0,0.035610,0.008902,168.803693,1.432723e-57
wri,1.0,0.308907,0.308907,5857.335951,5.893477e-132
vulnerability,1.0,0.020322,0.020322,385.340853,3.086974e-45
susceptibility,1.0,0.000033,0.000033,0.624905,4.303517e-01
lack_of_coping_capabilities,1.0,0.001762,0.001762,33.410918,3.575352e-08
lack_of_adaptive_capacities,1.0,0.000051,0.000051,0.969567,3.262127e-01


Nos hemos dado cuenta que las columnas con la que nos quedamos son las misma del dataframe df ya que los resultados son similares

In [16]:
lm_outliers.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       exposure_Sklearn   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     1976.
Date:                Mon, 30 Jan 2023   Prob (F-statistic):          2.16e-205
Time:                        19:18:46   Log-Likelihood:                 764.47
No. Observations:                 211   AIC:                            -1441.
Df Residuals:                     167   BIC:                            -1293.
Df Model:                          43                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               -0.6151      0.578     -1.064      0.289      -1.757       0.527
region[T.bangladesh]                    -0.0835      0.012     -6.706      0.000      -0.108      -0.059
region[T.brunei darussalam]             -0.0661      0.008     -7.813      0.000      -0.083      -0.049
region[T.cambodia]                      -0.0851      0.013     -6.394      0.000      -0.111      -0.059
region[T.cape verde]                    -0.0734      0.011     -6.903      0.000      -0.094      -0.052
region[T.chile]                         -0.0973      0.008    -11.686      0.000      -0.114      -0.081
region[T.costa rica]                    -0.0705      0.007     -9.917      0.000      -0.084      -0.056
region[T.djibouti]                      -0.0780      0.014     -5.486      0.000      -0.106      -0.050
region[T.dominica]                      -0.0128      0.009     -1.386      0.168      -0.031       0.005
region[T.east timor]                    -0.0777      0.014     -5.499      0.000      -0.106      -0.050
region[T.el salvador]                   -0.0719      0.009     -7.800      0.000      -0.090      -0.054
region[T.fiji]                          -0.0719      0.009     -8.027      0.000      -0.090      -0.054
region[T.guatemala]                     -0.0825      0.011     -7.591      0.000      -0.104      -0.061
region[T.guyana]                        -0.0575      0.010     -5.953      0.000      -0.077      -0.038
region[T.jamaica]                       -0.0832      0.010     -8.633      0.000      -0.102      -0.064
region[T.japan]                          0.1104      0.006     18.785      0.000       0.099       0.122
region[T.katar]                         -0.0029      0.004     -0.779      0.437      -0.010       0.004
region[T.kiribati]                      -0.0861      0.014     -6.363      0.000      -0.113      -0.059
region[T.mauritius]                     -0.0791      0.008     -9.477      0.000      -0.096      -0.063
region[T.netherlands]                    0.0554      0.007      8.256      0.000       0.042       0.069
region[T.nicaragua]                     -0.0871      0.011     -7.597      0.000      -0.110      -0.064
region[T.papua new guinea]              -0.0845      0.017     -4.990      0.000      -0.118      -0.051
region[T.philippines]                   -0.0746      0.009     -7.862      0.000      -0.093      -0.056
region[T.qatar]                         -0.0251      0.004     -6.601      0.000      -0.033      -0.018
region[T.solomon islands]               -0.0862      0.013     -6.612      0.000      -0.112      -0.060
region[T.south korea]                   -0.2173      0.015    -14.544      0.000      -0.247      -0.188
region[T.tonga]                       